In [21]:
import matplotlib.pyplot as plt
import numpy as np
import random
import os
from tqdm import tqdm

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
from torch import nn
import time
import cv2

## 0 Load and Pre-process Dataset

In [22]:
eybpath = './CroppedYale'
subjects = os.listdir(eybpath)
IMG = np.zeros((38, 64, 32, 32))
for ii, s in enumerate(subjects):  # 38 subjects
    subpath = eybpath + '/' + s
    imgs = os.listdir(subpath)

    for kk, im in enumerate(imgs):  # 64 images in each subject file
        impath = subpath + '/' + im
        img = cv2.imread(impath, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, dsize=(32,32))
        # Normalization
        img = img / 255
        IMG[ii][kk] = img
# IMG to tensor
IMG = torch.from_numpy(IMG) # torch.Size([38, 64, 32, 32])

In [23]:
# Obtain training set, validation set and testing set
# Split dataset randomly
list_sub = list(range(0, 64))
num_train = 52
num_val = 6
num_test = 6    # Each subject has 52 training images, 6 validation images, and 6 testing images

# train data: torch.Size([38, 52, 32, 32])
m = random.sample(range(0, 64), num_train)
train_data = torch.zeros((IMG.shape[0], num_train, IMG.shape[2], IMG.shape[3]))
for tt, mm in enumerate(m):
    list_sub.remove(mm)
    train_data[:,tt,:,:] = IMG[:,mm,:,:]
# val data: torch.Size([38, 6, 32, 32])
val_data = torch.zeros((IMG.shape[0], num_val, IMG.shape[2], IMG.shape[3]))
k = random.sample(list_sub, num_val)
for rr, kk in enumerate(k):
    list_sub.remove(kk)
    val_data[:,rr,:,:] = IMG[:,kk,:,:]
# test data: torch.Size([38, 6, 32, 32])
test_data = torch.zeros((IMG.shape[0], num_test, IMG.shape[2], IMG.shape[3]))
for uu, ll in enumerate(list_sub):
    test_data[:,uu,:,:] = IMG[:,ll,:,:]

In [24]:
def split_to_batch(data, batch_size):
    data_all = data.reshape((data.shape[0]*data.shape[1], data.shape[2], data.shape[3]))
    num_batch = int(np.ceil(data_all.shape[0] / batch_size))
    data_batches = []

    for i in range(num_batch):
        if i != num_batch-1:
            data_batch = data_all[i*batch_size: (i+1)*batch_size,:,:] # torch.Size([#Batch, 32, 32])
            #data_batch = data_batch.reshape(data_batch.shape[0], 1, data_batch.shape[1], data_batch.shape[2])
            data_batch = data_batch.unsqueeze(1)
            data_batches.append(data_batch)
        else:
            data_batch = data_all[i*batch_size: ,:,:]
            #data_batch = data_batch.reshape(data_batch.shape[0], 1, data_batch.shape[1], data_batch.shape[2])
            data_batch = data_batch.unsqueeze(1)
            data_batches.append(data_batch)

    return data_batches

In [25]:
# Split dataset into batches
batch_size = 32
train_batches = split_to_batch(train_data, batch_size)
val_batches = split_to_batch(val_data, batch_size)
test_batches = split_to_batch(test_data, batch_size)

## 1 Define Encoder and Decoder classes

In [26]:
class Encoder(nn.Module):

    def __init__(self, encoded_space_dim, fc2_input_dim):
        super().__init__()

        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(8, 16, 3, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=0),
            nn.ReLU(),
        )

        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)

        ### Linear section
        self.encoder_lin = nn.Sequential(
            nn.Linear(3 * 3 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, encoded_space_dim)
        )

    def forward(self, x):
        # Apply convolutions
        x = self.encoder_cnn(x)
        # Flatten
        x = self.flatten(x)
        # Apply linear layers
        x = self.encoder_lin(x)
        return x

In [27]:
class Decoder(nn.Module):

    def __init__(self, encoded_space_dim, fc2_input_dim):
        super().__init__()

        ### Linear section
        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 3 * 3 * 32),
            nn.ReLU()
        )

        ### Unflatten
        self.unflatten = nn.Unflatten(dim=1, unflattened_size=(32, 3, 3))

        ### Convolutional section
        self.decoder_conv = nn.Sequential(
            # First transposed convolution
            nn.ConvTranspose2d(32, 16, 3, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 8, 3, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 1, 4, stride=2)
        )

    def forward(self, x):
        # Apply linear layers
        x = self.decoder_lin(x)
        # Unflatten
        x = self.unflatten(x)
        # Apply transposed convolutions
        x = self.decoder_conv(x)
        # Apply a sigmoid to force the output to be between 0 and 1 (valid pixel values)
        x = torch.sigmoid(x)
        return x

## 2 Train model

In [28]:
### Training function
def train_epoch_den(encoder, decoder, device, dataloader, optimizer, val_lambda0=1, val_lambda=1, val_lambdaprime=1):
    # Set train mode for both the encoder and the decoder
    encoder.train()
    decoder.train()
    train_loss1 = []
    train_loss2 = []
    train_loss3 = []
    train_loss = []

    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for image_batch in dataloader: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        # Move tensor to the proper device
        image_batch = image_batch.to(device)
        # Encode data (z)
        encoded_data = encoder(image_batch)
        z_zeros = torch.zeros_like(encoded_data)
        # Decode data (x_hat)
        decoded_data = decoder(encoded_data)
        # Re-encode data (z_hat)
        z_hat = encoder(decoded_data)

        # Evaluate loss
        loss_fn = torch.nn.MSELoss()
        loss1 = loss_fn(image_batch, decoded_data)
        loss2 = loss_fn(encoded_data, z_hat)
        loss3 = loss_fn(encoded_data, z_zeros)
        loss = val_lambda0 * loss1 + val_lambda * loss2 + val_lambdaprime * loss3

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Batch loss
        train_loss1.append(loss1.detach().cpu().numpy())
        train_loss2.append(loss2.detach().cpu().numpy())
        train_loss3.append(loss3.detach().cpu().numpy())
        train_loss.append(loss.detach().cpu().numpy())

    return np.mean(train_loss1), np.mean(train_loss2), np.mean(train_loss3), np.mean(train_loss)

In [29]:
### Testing function
def test_epoch_den(encoder, decoder, device, dataloader, loss_fn):
    # Set evaluation mode for encoder and decoder
    encoder.eval()
    decoder.eval()
    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        conc_zeros = []
        for image_batch in dataloader:
            # Move tensor to the proper device
            image_batch = image_batch.to(device)
            x_zeros = torch.zeros_like(image_batch)
            # Encode data
            encoded_data = encoder(image_batch)
            # Decode data
            decoded_data = decoder(encoded_data)
            # Append the network output and the original image to the lists
            conc_out.append(decoded_data.cpu())
            conc_label.append(image_batch.cpu())
            conc_zeros.append(x_zeros.cpu())
        # Create a single tensor with all the values in the lists
        conc_out = torch.cat(conc_out)
        conc_label = torch.cat(conc_label)
        conc_zeros = torch.cat(conc_zeros)
        # Evaluate global loss
        lossx = loss_fn(conc_out, conc_label)
        Ex = loss_fn(conc_out, conc_zeros)
        Ex = Ex.detach().cpu().numpy()
        lossx = lossx.detach().cpu().numpy()
        # Calculate SNR_x
        snr_x = 10 * np.log10(Ex / lossx)

    return lossx, snr_x

#### Model of different dimensions

In [ ]:
snrs_x = []
rec_batch = []
v_loss = []
### Initialize the two networks
for d in range(4, 101, 4):
    print('d = %d: ' %d)
    ### Set the random seed for reproducible results
    torch.manual_seed(0)
    encoder = Encoder(encoded_space_dim=d, fc2_input_dim=128)
    decoder = Decoder(encoded_space_dim=d, fc2_input_dim=128)
    ### Define the loss function
    loss_fn = torch.nn.MSELoss()

    ### Define an optimizer (both for the encoder and the decoder!)
    lr = 0.001 # Learning rate

    params_to_optimize = [
    {'params': encoder.parameters()},
    {'params': decoder.parameters()}
    ]

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    optim = torch.optim.Adam(params_to_optimize, lr=lr)

    # Move both the encoder and the decoder to the selected device
    encoder.to(device)
    decoder.to(device)

    ### Setting parameters
    num_epochs = 30
    val_lambda0 = 1
    val_lambda = 1
    val_lambdaprime = 1
    loss = []

    for epoch in range(num_epochs):
        ### Training
        train_loss1, train_loss2, train_loss3, train_loss=train_epoch_den(
            encoder=encoder,
            decoder=decoder,
            device=device,
            dataloader=train_batches,
            optimizer=optim,
            val_lambda0=val_lambda0,
            val_lambda=val_lambda,
            val_lambdaprime=val_lambdaprime)

        loss.append(train_loss)
        # Print loss
        print('EPOCH {}/{} \t train loss {:.3f}'.format(epoch+1, num_epochs, train_loss))

        if epoch==num_epochs-1:
            ### Validation
            val_loss, snr_x = test_epoch_den(
            encoder=encoder,
            decoder=decoder,
            device=device,
            dataloader=val_batches,
            loss_fn=loss_fn)

            snrs_x.append(snr_x)
            v_loss.append(val_loss)

            prefix = './model/dimension' + '/CAE_dim{}'.format(d)+ '_lmd_{}_{}_{}'.format(val_lambda0, val_lambda, val_lambdaprime) + '_epc{}_'.format(epoch+1)
            name = time.strftime(prefix + '%m%d_%H_%M_%S.pth')
            torch.save([encoder, decoder], name)

    img = test_data[10][5].unsqueeze(0).unsqueeze(0).to(device)
    encoder.eval()
    decoder.eval()
    with torch.no_grad():
        rec_img = decoder(encoder(img))
    rec_batch.append(rec_img.cpu().squeeze().numpy())

## 3 Results

### 3.1 SNR_x vs. Dimension d

In [ ]:
### SNR_x-d
snrs_x = np.array(snrs_x)
dx = np.linspace(4, 100, 25)
plt.plot(dx, snrs_x, marker='o')
plt.xlabel('d')
plt.ylabel('SNR/dB')
plt.savefig('./figs/d_SNR_x.png')
plt.show()

### 3.2 Validation Loss vs. Dimension d

In [ ]:
### Val_loss-d
plt.plot(dx, v_loss, marker='o')
plt.xlabel('d')
plt.ylabel('Validation Loss')
plt.savefig('./figs/d_val_loss.png')
plt.show()

### 3.3 Reconstructed Images vs. Dimension d

In [ ]:
### Rec_img-d
for idx, img in enumerate(rec_batch):
    ax = plt.subplot(5, 5, idx+1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title('d=%d' %dx[idx], fontsize='small')
    plt.imshow(img, cmap='gist_gray')

plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.7,
                    top=0.9,
                    wspace=0.3,
                    hspace=0.6)
plt.savefig('./figs/d_Rec_imgs.png')
plt.show()